<a href="https://colab.research.google.com/github/EllisBuxton/PPIT-Project/blob/main/MusicGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Genetic Alogirthm Melody Generation**

In this notebook I will be creating a Genetic Algorithm and using it to create a melody.

I hope to show the following :    

1.   What are Genetic Algorithm's.
2.   How they work.
3.   Generating Melody's using the Genetic Algorithm





# Setup

The **random** module is used in this genetic algorithm implementation to introduce randomness at various stages, such as generating random genomes, selecting individuals from the population, performing mutation, and determining crossover points. Randomness is a crucial aspect of genetic algorithms because it helps explore the search space effectively, preventing the algorithm from getting stuck in local optima and promoting diversity within the population.

In [ ]:
from random import choices, randint, randrange, random, sample

The **typing** module is used to provide type hints for function parameters and return values. Type hints improve code readability and maintainability by indicating the expected types of arguments and return values for functions.

In [ ]:
from typing import List, Optional, Callable, Tuple, Dict

The **os** Module is used to provide access to operating system functionalities. In this script, it's used for creating directories and handling file paths

In [ ]:
import os

The **time** module provides access to time-related functions. It's used for introducing delays in the script, such as waiting before playing the next generation of melodies.

In [ ]:
import time

The **datetime** module is used for generating unique folder names based on the current timestamp.

In [ ]:
import datetime

The **midiutil** module is used for generating MIDI files based on the melodies generated by the genetic algorithm.

In [ ]:
import midiutil

The **pyo** module is a Python library for digital signal processing, synthesis, and audio programming. It's used in this script for sound synthesis, generating metronome sounds, and playing melodies.

In [ ]:
import pyo

**MidiUtil** and **Pyo** may need to be installed using pip.

In [ ]:
!pip install pyo
!pip install midiutil

This notebook will be split into 2 parts, I will go into detail on the Genetic Algorithm itself and then explain how i used it to create melody's

# The Algorithm:

In [7]:

# Type Aliases (better maintainability)
Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

# Function to generate a random genome of specified length
def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)

# Function to generate a population of genomes
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

# Function to perform crossover between two genomes
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of the same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

# Function to perform mutation on a genome
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

# Function to calculate the total fitness of a population
def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum(fitness_func(genome) for genome in population)

# Function to select a pair of genomes from the population for reproduction
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )

# Function to generate a weighted distribution for selection
def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene) + 1)

    return result

# Function to sort the population based on fitness
def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)

# Function to convert a genome to a string for printing
def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

# Function to print statistics of the current population
def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join(genome_to_string(gene) for gene in population))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

# Function to run the genetic algorithm
def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i


In [ ]:
import os
import random
import time
from datetime import datetime
from typing import Dict, List

from midiutil import MIDIFile
from pyo import *

# Constants
BITS_PER_NOTE = 4
KEYS = ["C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#", "Ab", "A", "A#", "Bb", "B"]
SCALES = ["major", "minorM", "dorian", "phrygian", "lydian", "mixolydian", "majorBlues", "minorBlues"]


# Helper function to convert a list of bits to an integer
def int_from_bits(bits: List[int]) -> int:
    return int(sum([bit*pow(2, index) for index, bit in enumerate(bits)]))


# Function to prompt for user input with optional default value and type casting
def prompt_for_input(prompt: str, default=None, type_=None):
    user_input = input(prompt + (f" [{default}]" if default is not None else "") + ": ")
    if user_input == "" and default is not None:
        return default
    if type_ is not None:
        try:
            return type_(user_input)
        except ValueError:
            print("Invalid input. Please try again.")
            return prompt_for_input(prompt, default, type_)
    return user_input


# Function to convert a genome to melody representation
def genome_to_melody(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: int, key: str, scale: str, root: int) -> Dict[str, list]:
    # Extract notes from the genome
    notes = [genome[i * BITS_PER_NOTE:i * BITS_PER_NOTE + BITS_PER_NOTE] for i in range(num_bars * num_notes)]

    note_length = 4 / float(num_notes)

    # Define the scale
    scl = EventScale(root=key, scale=scale, first=root)

    melody = {
        "notes": [],
        "velocity": [],
        "beat": []
    }

    # Convert notes to melody representation
    for note in notes:
        integer = int_from_bits(note)

        if pauses:
            integer = int(integer % pow(2, BITS_PER_NOTE - 1))

        if integer >= pow(2, BITS_PER_NOTE - 1):
            melody["notes"] += [0]
            melody["velocity"] += [0]
            melody["beat"] += [note_length]
        else:
            if len(melody["notes"]) > 0 and melody["notes"][-1] == integer:
                melody["beat"][-1] += note_length
            else:
                melody["notes"] += [integer]
                melody["velocity"] += [127]
                melody["beat"] += [note_length]

    steps = []
    for step in range(num_steps):
        steps.append([scl[(note+step*2) % len(scl)] for note in melody["notes"]])

    melody["notes"] = steps
    return melody


# Function to convert a genome to events for sound synthesis
def genome_to_events(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: bool, key: str, scale: str, root: int, bpm: int) -> [Events]:
    melody = genome_to_melody(genome, num_bars, num_notes, num_steps, pauses, key, scale, root)

    return [
        Events(
            midinote=EventSeq(step, occurrences=1),
            midivel=EventSeq(melody["velocity"], occurrences=1),
            beat=EventSeq(melody["beat"], occurrences=1),
            attack=0.001,
            decay=0.05,
            sustain=0.5,
            release=0.005,
            bpm=bpm
        ) for step in melody["notes"]
    ]


# Function to evaluate the fitness of a genome
def fitness(genome: Genome, s: Server, num_bars: int, num_notes: int, num_steps: int,
            pauses: bool, key: str, scale: str, root: int, bpm: int) -> int:
    m = metronome(bpm)

    events = genome_to_events(genome, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
    for e in events:
        e.play()
    s.start()

    rating = input("Rating (0-5)")

    for e in events:
        e.stop()
    s.stop()
    time.sleep(1)

    try:
        rating = int(rating)
    except ValueError:
        rating = 0

    return rating


# Function to generate a metronome sound
def metronome(bpm: int):
    met = Metro(time=1 / (bpm / 60.0)).play()
    t = CosTable([(0, 0), (50, 1), (200, .3), (500, 0)])
    amp = TrigEnv(met, table=t, dur=.25, mul=1)
    freq = Iter(met, choice=[660, 440, 440, 440])
    return Sine(freq=freq, mul=amp).mix(2).out()


# Function to save the melody generated from a genome to MIDI file
def save_genome_to_midi(filename: str, genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                        pauses: bool, key: str, scale: str, root: int, bpm: int):
    melody = genome_to_melody(genome, num_bars, num_notes, num_steps, pauses, key, scale, root)

    if len(melody["notes"][0]) != len(melody["beat"]) or len(melody["notes"][0]) != len(melody["velocity"]):
        raise ValueError

    mf = MIDIFile(1)

    track = 0
    channel = 0

    time = 0.0
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, bpm)

    for i, vel in enumerate(melody["velocity"]):
        if vel > 0:
            for step in melody["notes"]:
                mf.addNote(track, channel, step[i], time, melody["beat"][i], vel)

        time += melody["beat"][i]

    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "wb") as f:
        mf.writeFile(f)


# Main function
def main():
    # Prompt user for various parameters
    num_bars = prompt_for_input("Number of bars", default=8, type_=int)
    num_notes = prompt_for_input("Notes per bar", default=4, type_=int)
    num_steps = prompt_for_input("Number of steps", default=1, type_=int)
    pauses = prompt_for_input("Introduce Pauses (True/False)", default=False, type_=bool)
    key = prompt_for_input("Key", default="C")
    scale = prompt_for_input("Scale", default="major")
    root = prompt_for_input("Scale Root", default=4, type_=int)
    population_size = prompt_for_input("Population size", default=10, type_=int)
    num_mutations = prompt_for_input("Number of mutations", default=2, type_=int)
    mutation_probability = prompt_for_input("Mutation probability", default=0.5, type_=float)
    bpm = prompt_for_input("BPM", default=128, type_=int)

    folder = str(int(datetime.now().timestamp()))

    # Generate initial population
    population = [generate_genome(num_bars * num_notes * BITS_PER_NOTE) for _ in range(population_size)]

    s = Server().boot()

    population_id = 0

    running = True
    while running:
        # Shuffle the population
        random.shuffle(population)

        # Evaluate fitness for each genome in the population
        population_fitness = [(genome, fitness(genome, s, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)) for genome in population]

        # Sort the population based on fitness
        sorted_population_fitness = sorted(population_fitness, key=lambda e: e[1], reverse=True)

        # Select top genomes for next generation
        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):

            # Function to look up fitness of a genome
            def fitness_lookup(genome):
                for e in population_fitness:
                    if e[0] == genome:
                        return e[1]
                return 0

            # Selection, crossover, and mutation
            parents = selection_pair(population, fitness_lookup)
            offspring_a, offspring_b = single_point_crossover(parents[0], parents[1])
            offspring_a = mutation(offspring_a, num=num_mutations, probability=mutation_probability)
            offspring_b = mutation(offspring_b, num=num_mutations, probability=mutation_probability)
            next_generation += [offspring_a, offspring_b]

        print(f"population {population_id} done")

        # Playback top two genomes
        events = genome_to_events(population[0], num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        for e in events:
            e.play()
        s.start()
        input("here is the no1 hit …")
        s.stop()
        for e in events:
            e.stop()

        time.sleep(1)

        events = genome_to_events(population[1], num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        for e in events:
            e.play()
        s.start()
        input("here is the second best …")
        s.stop()
        for e in events:
            e.stop()

        time.sleep(1)

        # Save MIDI files for each genome in the population
        print("saving population midi …")
        for i, genome in enumerate(population):
            save_genome_to_midi(f"{folder}/{population_id}/{scale}-{key}-{i}.mid", genome, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        print("done")

        # Prompt user to continue or stop
        running = input("continue? [Y/n]") != "n"
        population = next_generation
        population_id += 1


if __name__ == '__main__':
    main()
